# Run Aggregation

Use the cell below to run the full aggregation pipeline for HERD. This will take your prompt, route it to the relevant experts, and return your final aggregated response.

In [ ]:
import requests

API_URL = "http://localhost:8000/Call_experts_locally_and_aggregate"

prompt = input("Enter your question: ")

payload = {
    "text": prompt,
    "top_k": 3
}

try:
    response = requests.post(API_URL, json=payload, timeout=60)
    response.raise_for_status()
    data = response.json()
    
    print("\nFinal Answer:")
    print(data.get("final_answer", "[No final answer returned]"))

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except Exception as e:
    print(f"unexpected error: {e}")
